In [1]:
%run ./read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-f33c4ef1-0717-4275-83bf-ea589347711c;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (51ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (32ms)
:: resolution report :: resolve 717ms :: artifacts dl 86ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	--------

23/03/14 04:52:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/14 04:52:38 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [18]:
%pip install tensorflow --quiet
%pip install tensorflow_gnn --quiet
%pip install tensorflow_io --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Note: you may need to restart the kernel to use updated packages.


In [27]:
import numpy as np
import tensorflow as tf
import tensorflow_io as tfio
from tensorflow import keras
import tensorflow_gnn as tfgnn
from tensorflow_gnn import runner
from tensorflow.keras import layers
import matplotlib.pyplot as plt

In [40]:
graph_schema = tfgnn.read_schema("./graph_schema.pbtxt")
gtspec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)

In [41]:
gtspec

GraphTensorSpec({'context': ContextSpec({'features': {'malicious': RaggedTensorSpec(TensorShape([1, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'node_sets': {'actorID': NodeSetSpec({'features': {'parent_image_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None), 'objectID': NodeSetSpec({'features': {'base_address': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32), 'file_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32), 'image_path': RaggedTensorSpec(TensorShape([None, None]), tf.string, 1, tf.int32)}, 'sizes': TensorSpec(shape=(1,), dtype=tf.int32, name=None)}, TensorShape([]), tf.int32, None)}, 'edge_sets': {'id': EdgeSetSpec({'features': {'event_day': TensorSpec(shape=(None,), dtype=tf.int64, name=None), 'event_hour': TensorSpec(shape=(None

In [37]:
train_ds_provider = runner.TFRecordDatasetProvider(file_pattern="s3://sapient-bucket-trusted/prod/graph/first_events/")

In [38]:
valid_ds_provider = runner.TFRecordDatasetProvider(file_pattern="s3://sapient-bucket-trusted/prod/graph/first_events/")

In [53]:
graph_tensor_spec = tfgnn.create_graph_spec_from_schema_pb(graph_schema)
dataset = dataset.map(
    lambda serialized: tfgnn.parse_single_example(graph_tensor_spec, serialized))

for i, graph in enumerate(dataset.take(3)):
  print(f"Input {i}: {graph}")

NameError: name 'dataset' is not defined

In [77]:
def getFirstEventFiles():
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(
        Bucket = 'sapient-bucket-trusted',
        Prefix = 'prod/graph/first_events')

    all_files = []
    for content in response.get('Contents', []):
        all_files.append(content['Key'])
        all_files = [f"sapient-bucket-trusted/" + f for f in content['Key'] if "parquet" in f]

    return all_files

In [78]:
getFirstEventFiles()

['prod/graph/first_events/_SUCCESS',
 'prod/graph/first_events/part-00000-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00001-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00002-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00003-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00004-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00005-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00006-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00007-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00008-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'prod/graph/first_events/part-00009-0d6be2ab-d553-4885-be78-abb9efceecc7-c000.snappy.parquet',
 'p

In [55]:
def map_fn(file_location):
    return tfio.IODataset.from_parquet(file_location, columns=columns)

In [ ]:
train_files = [ s3bucket + "/path/file_{}.parquet".format(i) for i in range(train_size) ]

In [ ]:
train_size = 640
batch_size = 32

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices((train_files, train_labels))

In [ ]:
files = [ s3bucket + "/path/file_{}.parquet".format(i) for i in range(5) ]
ds = tf.data.Dataset.from_tensor_slices(files).map(map_fn)